In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../../datasets/Alphabets Recognizer/A_Z Handwritten Data.csv')

In [4]:
df.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df['0'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [6]:
y = df['0']
X = df.drop('0',axis=1)

In [7]:
X.shape

(372450, 784)

In [8]:
#img = X.iloc[3,:].values.reshape(28,28)
#plt.imshow(img)

In [9]:
X = X/255 # scaling

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
ya = to_categorical(y,num_classes=26)
ya.shape

(372450, 26)

In [12]:
X.shape

(372450, 784)

In [14]:
X = X.values.reshape(372450,28,28,1)
X.shape

(372450, 28, 28, 1)

# Model Building

In [15]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten
from tensorflow.keras.models import Sequential

In [16]:
model = Sequential()
model.add(Conv2D(16,(3,3),activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D())
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(104,activation='relu'))
model.add(Dense(52,activation='relu'))
model.add(Dense(52,activation='relu'))
model.add(Dense(26,activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 32)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 104)               8

In [18]:
model.fit(X,ya,batch_size=128,epochs=10)

Epoch 1/10


2910/2910 [==============================] - 73s 23ms/step - loss: 0.2127 - accuracy: 0.9391
Epoch 2/10
2910/2910 [==============================] - 66s 23ms/step - loss: 0.0616 - accuracy: 0.9826
Epoch 3/10
2910/2910 [==============================] - 73s 25ms/step - loss: 0.0451 - accuracy: 0.9874
Epoch 4/10
2910/2910 [==============================] - 63s 22ms/step - loss: 0.0358 - accuracy: 0.9898
Epoch 5/10
2910/2910 [==============================] - 58s 20ms/step - loss: 0.0298 - accuracy: 0.9913
Epoch 6/10
2910/2910 [==============================] - 58s 20ms/step - loss: 0.0252 - accuracy: 0.9927
Epoch 7/10
2910/2910 [==============================] - 58s 20ms/step - loss: 0.0215 - accuracy: 0.9937
Epoch 8/10
2910/2910 [==============================] - 61s 21ms/step - loss: 0.0184 - accuracy: 0.9945
Epoch 9/10
2910/2910 [==============================] - 65s 22ms/step - loss: 0.0162 - accuracy: 0.9951
Epoch 10/10
2910/2910 [==============================] - 67s 2

In [19]:
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: 0, dtype: int64

In [20]:
img = X[0,:].reshape(1,28,28,1)
model.predict_on_batch(img).argmax()

0

In [22]:
model.predict_on_batch(X[:5,:]).argmax(axis=1)

array([0, 0, 0, 0, 0], dtype=int64)

# Predictions on own handwriting

In [23]:
import cv2

In [26]:
def get_alphabets(filename):
    path = '../../datasets/My A-Z Handwritten Alphabets/' + filename
    A = cv2.imread(path,0)
    A = cv2.resize(A,(28,28))
    A = A/255
    A = A.reshape(1,28,28,1)
    return model.predict_on_batch(A).argmax()

In [27]:
get_alphabets('A1.jpg')

0

In [28]:
import os

In [33]:
filenames = os.listdir('../../datasets/My A-Z Handwritten Alphabets/')
#filenames

In [42]:
#img = '../../datasets/My A-Z Handwritten Alphabets/P#1.JPG'#.reshape(28,28)
#plt.imshow(img)

In [43]:
sc = 0
for file in filenames:
    ya = file.split('.')[0][0]
    yp = chr(65+get_alphabets(file))
    if ya == yp:
        sc += 1
    print(file,'\t', yp)
print(f'Score : {sc}/{len(filenames)} OR {sc/len(filenames)}')

A1.jpg 	 A
A2.jpg 	 A
B1.jpg 	 B
B2.jpg 	 B
C1.jpg 	 C
C2.jpg 	 C
D1.jpg 	 D
D2.jpg 	 D
E1.jpg 	 E
E2.jpg 	 E
F1.jpg 	 F
F2.jpg 	 F
G1.jpg 	 G
G2.jpg 	 G
H1.jpg 	 H
H2.jpg 	 H
I1.jpg 	 I
I2.jpg 	 I
J1.jpg 	 J
J2.jpg 	 J
K1.jpg 	 K
K2.jpg 	 K
L1.jpg 	 L
L2.jpg 	 L
M1.jpg 	 M
M2.jpg 	 M
N1.jpg 	 N
N2.jpg 	 N
O1.jpg 	 O
O2.jpg 	 O
P#1.jpg 	 Y
P1.jpg 	 P
P2.jpg 	 P
Q1.jpg 	 Q
Q2.jpg 	 Q
R1.jpg 	 R
R2.jpg 	 R
S#1.jpg 	 A
S1.jpg 	 S
S2.jpg 	 S
S3.jpg 	 S
S4.jpg 	 S
S5.jpg 	 S
S6.jpg 	 S
S7.jpg 	 S
T#1.jpg 	 Y
T1.jpg 	 T
T2.jpg 	 T
T3.jpg 	 T
T4.jpg 	 T
T5.jpg 	 T
U1.jpg 	 U
U2.jpg 	 U
V1.jpg 	 V
V2.jpg 	 V
W1.jpg 	 W
W2.jpg 	 W
X1.jpg 	 X
X2.jpg 	 X
Y1.jpg 	 Y
Y2.jpg 	 Y
Z1.jpg 	 Z
ZZ.jpg 	 Z
Score : 60/63 OR 0.9523809523809523


In [ ]:
# 1st = Score : 52/60 OR 0.8666666666666667
# 2nd = Score : 58/60 OR 0.9666666666666667
# CNN = Score : 60/60 OR 1.0